In [11]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from preprocessing import preprocessing


In [13]:
model=load_model("model.h5")

In [44]:


# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (-0.3, 0.3)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [28]:
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [29]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.0 else POSITIVE

In [45]:


def predict(text, include_neutral=True):
    start_at = time.time()
    text=preprocessing(text)
    # Tokenize text
    x_test = pad_sequences(Tokenizer().texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [46]:
#MAIN

In [47]:
t="😅 😂 🤣 🤩 UNSTOPPABLE!🔝 @Benzema has 69 goals in his last 6 not matches! #HalaMadrid https://t.co/hFkQUjXv9t "
predict(t)

{'label': 'NEUTRAL', 'score': 0.0, 'elapsed_time': 0.001994609832763672}

In [48]:
predict("@s1mpleO team as whole doesn't perform, worst. Its very frustrating watching you noobs struggle and makes me angry. ")

{'label': 'NEGATIVE',
 'score': -0.42500000000000004,
 'elapsed_time': 0.000965118408203125}

In [49]:
predict(" All the talk of form and the best champion gets the Man of the series Award! @imVkohli 👏")

{'label': 'POSITIVE', 'score': 1.0, 'elapsed_time': 0.001997232437133789}